In [ ]:
#Import modules
#Some of these aren't used, but got included here just in case (would need testing to remove unnecessary modules)
import numpy as np
import pandas as pd
import csv
from datetime import datetime
import datetime
import os
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import export_graphviz
from dmba import plotDecisionTree
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation
from sklearn.metrics import max_error,mean_absolute_error,mean_squared_log_error, mean_squared_error, r2_score
from flaml import AutoML
from flaml.ml import sklearn_metric_loss_score
from flaml.data import get_output_from_log
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
#pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [ ]:
#Read in CSV
full_df = pd.read_csv(r'<path to file>\Full_3.csv')

In [ ]:
#Dropping less columns than before
full_df.drop(columns={'blurb','country','name','CreatedDate','DeadLineDate','LaunchedDate','id','CreatorID','backers_count','pledged'},axis=1,inplace=True,errors='ignore')

In [ ]:
#Only run this next set of codeblocks once
#Changing goal and pledged amounts to account for inflation
dictInflation = {2008:1.268,2009:1.273,2010:1.252,2011:1.214,2012:1.189,2013:1.172,2014:1.153,2015:1.152,2016:1.137,2017:1.114,2018:1.087,2019:1.068,2020:1.055,2021:1}

In [ ]:
#Create an inflation adjusted column
full_df['usd_pledged_21'] = 0
for index, row in full_df.iterrows():
    print(index)
    full_df.loc[index,'usd_pledged_21'] = round(dictInflation[row['DeadLineDate_Year']]*row['usd_pledged'],2)

In [ ]:
#Drop the original pledged column
full_df.drop(columns={'usd_pledged'},axis=1,inplace=True,errors='ignore')
full_df['usd_pledged_21'] = full_df['usd_pledged_21'].astype(int)

In [ ]:
#Rank the projects in each category/subcategory
#This will help with categorization
new_df = pd.DataFrame()
for name, group in full_df.groupby(['CatID', 'SubCatID']):
    part_df = full_df[(full_df.CatID == name[0])&(full_df.SubCatID==name[1])]
    percRank = part_df.usd_pledged_21.rank(pct=True)
    part_df['PercentileRank'] = percRank
    
    print(name)
    print(len(part_df))
    
    for index,row in part_df.iterrows():
        rankVal = row['PercentileRank']
        if rankVal <= 0.1:
            part_df.loc[index,'Tens'] = 10
        elif rankVal <= 0.2:
            part_df.loc[index,'Tens'] = 20
        elif rankVal <= 0.3:
            part_df.loc[index,'Tens'] = 30
        elif rankVal <= 0.4:
            part_df.loc[index,'Tens'] = 40
        elif rankVal <= 0.5:
            part_df.loc[index,'Tens'] = 50
        elif rankVal <= 0.6:
            part_df.loc[index,'Tens'] = 60
        elif rankVal <= 0.7:
            part_df.loc[index,'Tens'] = 70
        elif rankVal <= 0.8:
            part_df.loc[index,'Tens'] = 80
        elif rankVal <= 0.9:
            part_df.loc[index,'Tens'] = 90
        else:
            part_df.loc[index,'Tens'] = 100

        if rankVal <= 0.2:
            part_df.loc[index,'Twenties'] = 20
        elif rankVal <= 0.4:
            part_df.loc[index,'Twenties'] = 40
        elif rankVal <= 0.6:
            part_df.loc[index,'Twenties'] = 60
        elif rankVal <= 0.8:
            part_df.loc[index,'Twenties'] = 80
        else:
            part_df.loc[index,'Twenties'] = 100

        if rankVal <= 0.33:
            part_df.loc[index,'Third'] = 33
        elif rankVal <= 0.66:
            part_df.loc[index,'Third'] = 66
        else:
            part_df.loc[index,'Third'] = 100

        if rankVal <= 0.5:
            part_df.loc[index,'Half'] = 50
        else:
            part_df.loc[index,'Half'] = 100
            
    new_df = new_df.append(part_df,ignore_index=True)

In [ ]:
#Convert rank to brackets
part_df = full_df[full_df['CatID'].isnull()]
percRank = part_df.usd_pledged_21.rank(pct=True)
part_df['PercentileRank'] = percRank

for index,row in part_df.iterrows():
    rankVal = row['PercentileRank']
    if rankVal <= 0.1:
        part_df.loc[index,'Tens'] = 10
    elif rankVal <= 0.2:
        part_df.loc[index,'Tens'] = 20
    elif rankVal <= 0.3:
        part_df.loc[index,'Tens'] = 30
    elif rankVal <= 0.4:
        part_df.loc[index,'Tens'] = 40
    elif rankVal <= 0.5:
        part_df.loc[index,'Tens'] = 50
    elif rankVal <= 0.6:
        part_df.loc[index,'Tens'] = 60
    elif rankVal <= 0.7:
        part_df.loc[index,'Tens'] = 70
    elif rankVal <= 0.8:
        part_df.loc[index,'Tens'] = 80
    elif rankVal <= 0.9:
        part_df.loc[index,'Tens'] = 90
    else:
        part_df.loc[index,'Tens'] = 100

    if rankVal <= 0.2:
        part_df.loc[index,'Twenties'] = 20
    elif rankVal <= 0.4:
        part_df.loc[index,'Twenties'] = 40
    elif rankVal <= 0.6:
        part_df.loc[index,'Twenties'] = 60
    elif rankVal <= 0.8:
        part_df.loc[index,'Twenties'] = 80
    else:
        part_df.loc[index,'Twenties'] = 100

    if rankVal <= 0.33:
        part_df.loc[index,'Third'] = 33
    elif rankVal <= 0.66:
        part_df.loc[index,'Third'] = 66
    else:
        part_df.loc[index,'Third'] = 100

    if rankVal <= 0.5:
        part_df.loc[index,'Half'] = 50
    else:
        part_df.loc[index,'Half'] = 100

new_df = new_df.append(part_df,ignore_index=True)

In [ ]:
np.random.seed(500)

In [ ]:
#Start creating the corpus
Corpus = new_df
Corpus.blurb = Corpus.blurb.astype(str)
Corpus.name = Corpus.name.astype(str)

In [ ]:
# Step - a : Remove blank rows if any.
Corpus['name'].dropna(inplace=True)
# Step - b : Change all the text to lower case. This is required as python interprets 'dog' and 'DOG' differently
Corpus['name'] = [entry.lower() for entry in Corpus['name']]
# Step - c : Tokenization : In this each entry in the corpus will be broken into set of words
Corpus['name']= [word_tokenize(entry) for entry in Corpus['name']]
# Step - d : Remove Stop words, Non-Numeric and perfom Word Stemming/Lemmenting.
# WordNetLemmatizer requires Pos tags to understand if the word is noun or verb or adjective etc. By default it is set to Noun
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV
for index,entry in enumerate(Corpus['name']):
    print(index)
    # Declaring Empty List to store the words that follow the rules for this step
    Final_words = []
    # Initializing WordNetLemmatizer()
    word_Lemmatized = WordNetLemmatizer()
    # pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V) or something else.
    for word, tag in pos_tag(entry):
        # Below condition is to check for Stop words and consider only alphabets
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            Final_words.append(word_Final)
    # The final processed set of words for each iteration will be stored in 'text_final'
    Corpus.loc[index,'text_final_name'] = str(Final_words)

In [ ]:
#Create dataset
Corpus.to_csv(r'<path to file>\MainDataset\Full_Corpus.csv',index=False)